In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from scipy.optimize import minimize
from scipy.integrate import odeint
from tqdm import tqdm
from scipy.sparse import csr_matrix
import time
from scipy.optimize import basinhopping

In [25]:
pop_vector

array([1000.,    0.,    0.,    0., 1100.,    0.,    0.,    0., 1200.])

In [21]:
patch_a_timeseries = np.array([5, 6, 9, 12, 15, 19, 24, 30, 37, 46, 56, 68, 75, 90])
patch_b_timeseries = np.array([5, 7, 12, 18, 26, 37, 53, 76, 107, 150, 180, 250, 400, 500])
patch_c_timeseries = np.array([2, 2, 4, 5, 8, 11, 14, 19, 25, 33, 44, 58, 75, 98])

home_patches = np.array([0,4,8])

# X-axis (assuming time steps)
time_steps = np.arange(len(patch_a_timeseries))

def expand_array(short_array):
    long_array = np.zeros(9)  # Create a zero array of the desired size

    # Assign values at the correct positions
    long_array[np.arange(3) * (3 + 1)] = short_array
    return long_array



all_station_timeseries = np.array([
    patch_a_timeseries,
    patch_b_timeseries,
    patch_c_timeseries
])


pop_dict = {
    "Patch A": 1000,
    "Patch B": 1100,
    "Patch C": 1200
}

short_pop_vector = np.array(list(pop_dict.values()))
pop_vector = expand_array(short_pop_vector)

station_list = ["Patch A","Patch B","Patch C"]

p_matrix = np.array([
    [0.836, 0.014, 0, 0.014, 0.071, 0.011, 0, 0.011, 0.043],
    [0, 1.0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1.0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1.0, 0, 0, 0, 0, 0],
    [0.071, 0.009, 0, 0.009, 0.816, 0.016, 0, 0.016, 0.064],
    [0, 0, 0, 0, 0, 1.0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1.0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1.0, 0],
    [0.043, 0.005, 0, 0.005, 0.064, 0.027, 0, 0.027, 0.828]
])

sparse_p = csr_matrix(p_matrix)

Nj = pop_vector @ p_matrix

valid_patch_indices = np.array([0,1,3,4,5,7,8])

station_borough_list = ["Borough AB","Borough AB","Borough C"]

borough_list = ["Borough AB","Borough C"]

np.set_printoptions(suppress=True)









# Define function to process solution
def process_solution(solution,timesteps):
    # Grouping by station subpopulation
    S_sol = solution[:9]
    I_sol = solution[9:18]
    I_sol_total = solution[18:]

    return [S_sol,I_sol,I_sol_total]



def run_model_London(params):
    
    start_time = time.time()
    
    S0,I0,I_total0,beta,gamma,timesteps = params
    
    beta = elongate_full_beta(beta)
    
     # Ensure no dividing by zero
    for i in range(9):
        if Nj[i] == 0:
            Nj[i] = 1
    
    # Set up initial y0 vector
    y0 = np.concatenate((S0,I0,I_total0))
    sparse_p = csr_matrix(p_matrix)
    
    beta_p_matrix = sparse_p.multiply(beta[:, None])
    
    t = np.linspace(0,timesteps,num=timesteps)
    
    
    
    def sir(y,timestep):
        S = y[:9]
        I = y[9:18]
        I_total = y[18:]

        dSdt = np.zeros_like(S)
        dIdt = np.zeros_like(I)
        dI_totaldt = np.zeros_like(I_total)

        Ij = p_matrix.T @ I
        
        
        method = "vector"
        
        if method == "vector":

            # Compute Ij_div_Nj efficiently
            Ij_div_Nj = Ij / Nj  # Shape (63001,)
        
            # Vectorized computation of infection terms using sparse matrix multiplication
            infection_terms = beta_p_matrix.multiply(S[:, None])  # (63001, 63001)
            infection_terms = infection_terms @ Ij_div_Nj  # (63001,)
        
            # Update derivatives
            dSdt -= infection_terms
            dIdt += infection_terms
            

        else:
            # Essentially, for all i, all j, dSdt[i] += -beta[j]*p_matrix[i][j]*S[i]*Ij[j]/Nj[j]
            for i in valid_patch_indices:
                for j in valid_patch_indices:
                    dSdt[i] += -beta[j]*p_matrix[i][j]*S[i]*Ij[j]/Nj[j]
            dIdt -= dSdt
    
        dI_totaldt += dIdt

        # Add the gamma terms
        dIdt -= gamma * I
        # dSdt += gamma * I

        # Concatenate results
        dx = np.concatenate((dSdt, dIdt, dI_totaldt))

        return dx
    
    y_log = np.zeros((len(t)+1,27))
    y_log[0] = y0.copy()
    

    for step in range(len(t)):
        results = sir(y_log[step],step)
        y_log[step+1] = y_log[step] + results
    
    solution = process_solution(y_log.T,timesteps)
    
    end_time = time.time()  # End the timer
    elapsed_time = end_time - start_time
    # print(f"run_model_London took {elapsed_time:.4f} seconds")
    
    return solution
    

In [22]:
def expand_timeseries(short_timeseries):
    
    transposed = short_timeseries.T
    
    long_timeseries = np.zeros((14,9))
    for i in range(14):
        long_timeseries[i] = expand_array(transposed[i])
        
    long_timeseries = np.array(long_timeseries)
    
    return long_timeseries

def expand_array(short_array):
    long_array = np.zeros(9)  # Create a zero array of the desired size

    # Assign values at the correct positions
    long_array[np.arange(3) * (3 + 1)] = short_array
    return long_array



def compress_timeseries(long_timeseries):
    compressed = np.zeros((14, 3))  # Adjusted to match the original short array shape
    
    for i in range(14):
        compressed[i] = compress_array(long_timeseries[i])
        
    return compressed.T  # Transpose back to match original input shape

def compress_array(long_array):
    return long_array[np.arange(3) * (3 + 1)]  # Extract values from the expanded positions


def expand_betas(short_betas,long_length):
    
    long_betas = np.zeros(long_length)
    index = 0
    for i in valid_patch_indices:
        long_betas[i] = short_betas[index]
        index += 1
    
    return long_betas

def elongate_beta(short_beta):
    long_beta = np.zeros(9)
    index = 0
    for i in range(9):
        if i in home_patches:
            long_beta[i] = short_beta[index]
            index += 1
        else:
            long_beta[i] = short_beta[-1]
    return long_beta

def elongate_full_beta(short_beta):
    long_beta = np.zeros(9)
    long_beta[valid_patch_indices] = short_beta
    return long_beta


def sir_simulation_fit_class(full_timeseries, pop_dict, plot_option):
    start_time = time.time()
    
    # Load data
    working_timeseries = expand_timeseries(full_timeseries)
    
    short_pop_vector = np.array(list(pop_dict.values()))
    pop_vector = expand_array(short_pop_vector)
    
    day_list = np.arange(14)  # There are 14 days

    # Stack so each row is [day_index, corresponding 9 values]
    data = np.column_stack((day_list[:, None], working_timeseries))


    # Initial guess for beta
    # initial_guess = 0.2*np.ones(len(valid_patch_indices)) # Randomly distributed betas
    initial_guess = np.random.uniform(0.05, 0.9, len(valid_patch_indices))

    
    def callback(beta, res=None):  # Add a second optional argument
        # print(f'Current best beta: {beta}')
        a = 1+1

    # Define bounds for beta values
    beta_bounds = [(0.05, 0.9)] * len(valid_patch_indices)

    # Optimize beta with bounds
    result = minimize(lambda beta: error_function(beta, data, pop_dict), initial_guess, method='Powell', callback=callback, bounds=beta_bounds)


    

    beta_opt = result.x

    # print('Estimated parameter (beta):', beta_opt)
    # print('Final minimized error:', result.fun)

    # Total population
    N = pop_vector.copy()

    # Time span
    tspan = np.linspace(0, 14, 14)
    
    transit_multiplier = 1
    gamma = 1 / 5  # Fixed gamma value
    betas = beta_opt
    I0 = data[0][1:]
    S0 = pop_vector-data[0][1:]
    I_total_0 = data[0][1:]
    
    params = [S0,I0,I_total_0,betas,gamma,int(np.max(data[:, 0]))]

    # Solve SIR model

    y = run_model_London(params)[2]

    # Extract cumulative cases
    I_sum_model = compress_timeseries(y.copy().T)


    if plot_option == "plot":
        # Plot results
        for i in range(3):
            
            plt.figure()
            plt.plot(tspan, I_sum_model[i], 'r-', linewidth=2, label='Model Cumulative Cases')
            plt.scatter(data[:, 0], data[:, 1], c='k', s=50, label='Observed Cumulative Cases')

            plt.xlabel('Time (Days)')
            plt.ylabel('Number of Individuals')
            plt.title(f'Fitting for station {station_list[i]}')
            plt.legend()
            plt.show()
    
    end_time = time.time()
    # print("Total time:",end_time-start_time)
    
    return beta_opt


def error_function(beta, data, pop_dict):
    short_pop_vector = np.array(list(pop_dict.values()))
    pop_vector = np.zeros(9)  # Create a zero array of the desired size

    # Assign values at the correct positions
    pop_vector[np.arange(3) * (3 + 1)] = short_pop_vector
    
    betas = beta.copy()

    tspan = data[:, 0]  # Use observed time points
    timespan = tspan.shape[0]-1

    # Solve SIR model with cumulative cases
    gamma = 1 / 5  # Fixed gamma value
    
        
    I0 = data[0][1:]
    S0 = pop_vector-I0
    I_total_0 = data[0][1:]
    
    params = [S0,I0,I_total_0,betas,gamma,timespan]
    
    y = run_model_London(params)[2]

    # Extract cumulative cases
    I_sum_model = compress_timeseries(y.T)
    raw_real_data = compress_timeseries(data.T[1:].T)
    normalizer = compress_array(pop_vector)[:, np.newaxis]
    test_data = I_sum_model/normalizer
    real_data = raw_real_data/normalizer

    # Compute squared error between observed and modeled cumulative cases
    squared_error = np.sum((test_data - real_data) ** 2)
    
    lambda_reg = 1e-6
    penalty = lambda_reg * np.sum((beta - np.mean(beta))**2)
    squared_error += penalty

    return squared_error

# Run the function
betas0 = sir_simulation_fit_class(all_station_timeseries, pop_dict, "plot")
betas = elongate_beta(betas0)
betas

NameError: name 'expand_concise_betas' is not defined

In [12]:
home_patches = np.array([0,4,8])
travel_patches = np.array([1,3,5,7])

In [8]:
elongate_full_beta(betas0)

array([0.35760972, 0.46895792, 0.        , 0.4691559 , 0.70063527,
       0.46925021, 0.        , 0.46933183, 0.35037139])

In [7]:
betas0

array([0.35760972, 0.46895792, 0.4691559 , 0.70063527, 0.46925021,
       0.46933183, 0.35037139])

In [9]:
np.average(betas0)

np.float64(0.4693303189431857)

In [13]:
np.average(elongate_full_beta(betas0)[travel_patches])

np.float64(0.46917396262494465)

In [57]:
alphas = np.ones(9)*(1/5)


def making_NGM(p_matrix,N_array,betas,alphas):
    size = betas.shape[0]  # 6084 for full London system
    print(size)
    NGM = np.zeros([size,size])
    for i in range(size):
        for j in range(size):
            N_div_a = N_array[i]/alphas[j]
            k_sum = 0
            for k in range(size):
                l_sum = 0
                for l in range(size):
                    l_sum += N_array[l]*p_matrix[l][k]
                if l_sum != 0:
                    k_sum += betas[k]*p_matrix[i][k]*p_matrix[j][k]/l_sum
            NGM[i][j] = k_sum*N_div_a
    return NGM

NGM1 = making_NGM(p_matrix,pop_vector,betas,alphas)
NGM1

9


array([[1.30670458, 1.12090707, 0.        , 1.12091368, 0.35739214,
        0.43169611, 0.        , 0.4316936 , 0.1669445 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.39313136, 0.79264143, 0.        , 0.7926461 , 2.7545871 ,
        0.69071377, 0.        , 0.69070975, 0.34767083],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],


In [62]:
alphas = np.ones(9)*(1/5)


def expand_concise_betas(home_patches,travel_patches,concise_beta):
    print(concise_beta.shape)
    long_beta = np.zeros(9)
    long_beta[np.concatenate([home_patches,travel_patches])] = concise_beta
    
    return long_beta

from scipy.sparse import csr_matrix
from tqdm import tqdm

def making_NGM(sparse_p, N_array, betas, alphas):
    size = betas.shape[0]  

    print("Computing l_sums...")
    l_sums = sparse_p.T.dot(N_array)  # Efficient matrix-vector multiplication
    l_sums[l_sums == 0] = 1  # Avoid division by zero

    print("Computing k_sums...")
    k_sums = csr_matrix((size, size))  # Initialize sparse result matrix

    # Iterate over nonzero elements of sparse_p
    for k in tqdm(range(size), desc="Processing k_sums"):
        if l_sums[k] == 0:  # Skip empty columns
            continue
        k_sums += sparse_p[:, k].dot((betas[k] * sparse_p[:, k].T / l_sums[k]))

    print("Finalizing NGM computation...")
    NGM = (N_array[:, None] / alphas) * k_sums.toarray()  # Convert to dense if necessary

    print("NGM computation complete.")
    return NGM

NGM2 = making_NGM(sparse_p,pop_vector,betas,alphas)
NGM2

Computing l_sums...
Computing k_sums...


Processing k_sums: 100%|██████████| 9/9 [00:00<00:00, 827.22it/s]

Finalizing NGM computation...
NGM computation complete.


array([[1.30670458, 1.12090707, 0.        , 1.12091368, 0.35739214,
        0.43169611, 0.        , 0.4316936 , 0.1669445 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.39313136, 0.79264143, 0.        , 0.7926461 , 2.7545871 ,
        0.69071377, 0.        , 0.69070975, 0.34767083],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],


In [70]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs

# Ensure NGM is stored efficiently in CSR format
NGM_sparse = csr_matrix(NGM2)

# Compute only the largest eigenvalue using shift-invert mode
eigenvalues, eigenvectors = eigs(NGM_sparse, k=1, which="LM", tol=1e-6)

print("Computed Top Eigenvalue:", eigenvalues)


Computed Top Eigenvalue: [2.94111407+0.j]


In [7]:
# Parallel beta fitting

from joblib import Parallel, delayed

def error_function(beta, data, pop_dict):
    """Compute error in parallel using multiple beta evaluations."""
    
    # Expand population vector
    short_pop_vector = np.array(list(pop_dict.values()))
    pop_vector = expand_array(short_pop_vector)

    # Initial conditions
    I0 = data[0][1:]
    S0 = pop_vector - I0
    I_total_0 = np.zeros(9)
    gamma = 1 / 5
    timespan = data.shape[0] - 1

    params = [S0, I0, I_total_0, beta, gamma, timespan]
    
    # Run SIR model simulations in parallel
    y_list = Parallel()(
        delayed(run_model_London)(params) for _ in range(len(valid_patch_indices))
    )

    # Extract cumulative cases
    I_sum_model = compress_timeseries(y_list[0][2].T)
    raw_real_data = compress_timeseries(data.T[1:].T)
    
    # Normalize
    normalizer = compress_array(pop_vector)[:, np.newaxis]
    test_data = I_sum_model / normalizer
    real_data = raw_real_data / normalizer

    # Compute squared error
    squared_error = np.sum((test_data - real_data) ** 2)
    
    # Regularization penalty
    penalty = 1e-3 * np.sum((beta - np.mean(beta)) ** 2)
    
    return squared_error + penalty


from scipy.optimize import minimize

def sir_simulation_fit_class(full_timeseries, pop_dict, plot_option):
    """Fit the SIR model parameters using parallelized optimization."""
    start_time = time.time()
    
    # Expand time series
    working_timeseries = expand_timeseries(full_timeseries)
    
    # Set up data for fitting
    day_list = np.arange(14)  
    data = np.column_stack((day_list[:, None], working_timeseries))

    # Initial guess for beta
    initial_guess = np.random.uniform(0.2, 0.6, len(valid_patch_indices))

    # Define bounds for beta values
    beta_bounds = [(0.05, 0.9)] * len(valid_patch_indices)

    # Optimize beta with parallel error function
    result = minimize(
        lambda beta: error_function(beta, data, pop_dict), 
        initial_guess, 
        method='trust-constr',  
        bounds=beta_bounds,
        options={'maxiter': 1000, 'gtol': 1e-6}
    )

    beta_opt = result.x

    print('Optimized beta:', beta_opt)
    print('Final minimized error:', result.fun)
    
    end_time = time.time()
    print("Total time:",end_time-start_time)

    return beta_opt

sir_simulation_fit_class(all_station_timeseries, pop_dict, "plot")

Optimized beta: [0.34393933 0.47945321 0.4794389  0.7712417  0.47944539 0.47948074
 0.32118925]
Final minimized error: 0.0030887703219116885
Total time: 0.720421314239502


array([0.34393933, 0.47945321, 0.4794389 , 0.7712417 , 0.47944539,
       0.47948074, 0.32118925])